In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [79]:
df = pd.read_csv(r"D:\MBA\3rd Semester\MBAG-305-Machine Learning\Final Project\Financial Risk for Loan Approval.csv")
df.head()

,ApplicationDate,Age,AnnualIncome,CreditScore,EmploymentStatus,EducationLevel,Experience,LoanAmount,LoanDuration,MaritalStatus,...,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
0,01/01/2018,45,39948,617,Employed,Master,22,13152,48,Married,...,3329.000000,0.724972,11,126928,0.199652,0.227590,419.805992,0.181077,0,49.0
1,02/01/2018,38,39709,628,Employed,Associate,15,26045,48,Single,...,3309.083333,0.935132,3,43609,0.207045,0.201077,794.054238,0.389852,0,52.0
2,03/01/2018,47,40724,570,Employed,Bachelor,26,17627,36,Married,...,3393.666667,0.872241,6,5205,0.217627,0.212548,666.406688,0.462157,0,52.0
3,04/01/2018,58,69084,545,Employed,High School,34,37898,96,Single,...,5757.000000,0.896155,5,99452,0.300398,0.300911,1047.506980,0.313098,0,54.0
4,05/01/2018,37,103264,594,Employed,Associate,17,9184,36,Married,...,8605.333333,0.941369,5,227019,0.197184,0.175990,330.179141,0.070210,1,36.0


In [81]:
df = df.drop(["ApplicationDate", "MonthlyIncome","BaseInterestRate", "InterestRate", "TotalDebtToIncomeRatio", "CreditScore"], axis=1)

In [83]:
df.columns

Index(['Age', 'AnnualIncome', 'EmploymentStatus', 'EducationLevel',
       'Experience', 'LoanAmount', 'LoanDuration', 'MaritalStatus',
       'NumberOfDependents', 'HomeOwnershipStatus', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines',
       'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'BankruptcyHistory',
       'LoanPurpose', 'PreviousLoanDefaults', 'PaymentHistory',
       'LengthOfCreditHistory', 'SavingsAccountBalance',
       'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities',
       'UtilityBillsPaymentHistory', 'JobTenure', 'NetWorth',
       'MonthlyLoanPayment', 'LoanApproved', 'RiskScore'],
      dtype='object')

In [85]:
df.shape

(20000, 30)

In [89]:
X = df.drop("LoanApproved", axis=1)
y = df["LoanApproved"]

In [91]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
category_cols = X.select_dtypes("object").columns
for c in category_cols:
    X[c] = le.fit_transform(X[c].values)

In [93]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)

In [95]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14000 entries, 7183 to 19957
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         14000 non-null  int64  
 1   AnnualIncome                14000 non-null  int64  
 2   EmploymentStatus            14000 non-null  int32  
 3   EducationLevel              14000 non-null  int32  
 4   Experience                  14000 non-null  int64  
 5   LoanAmount                  14000 non-null  int64  
 6   LoanDuration                14000 non-null  int64  
 7   MaritalStatus               14000 non-null  int32  
 8   NumberOfDependents          14000 non-null  int64  
 9   HomeOwnershipStatus         14000 non-null  int32  
 10  MonthlyDebtPayments         14000 non-null  int64  
 11  CreditCardUtilizationRate   14000 non-null  float64
 12  NumberOfOpenCreditLines     14000 non-null  int64  
 13  NumberOfCreditInquiries     14000

In [97]:
ytest.shape

(6000,)

In [99]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
regressor.fit(Xtrain, ytrain)
y_pred = regressor.predict(Xtest)

In [100]:
ytest = ytest.astype("category")

In [101]:
y_pred
threshold = 0.5

# Categorize probabilities as 1 and 0
y_pred = (y_pred >= threshold).astype(int)

In [102]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score

# Assuming you have your true labels (y_true) and predicted labels (y_pred)
f1 = f1_score(ytest, y_pred)
cm = confusion_matrix(ytest, y_pred)
acc = accuracy_score(ytest, y_pred)
prec = precision_score(ytest, y_pred)
print("F1-score:", f1)
print("Confusion Matrix:\n", cm)
print("Accuracy:", acc)
print("Precision:", prec)

F1-score: 0.9837685250529287
Confusion Matrix:
 [[4560   22]
 [  24 1394]]
Accuracy: 0.9923333333333333
Precision: 0.9844632768361582


In [103]:
from sklearn.metrics import accuracy_score
regressor.score(Xtrain, ytrain)
regressor = round(regressor.score(Xtrain, ytrain) * 100, 2)
regressor

99.63

In [104]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X, y)
feature_importances = rf_model.feature_importances_
feature_names = X.columns 

In [105]:
print("Feature Importances:")
for feature, importance in zip(feature_names, feature_importances):
    print(f"{feature}: {importance:.3f}")

Feature Importances:
Age: 0.008
AnnualIncome: 0.186
EmploymentStatus: 0.001
EducationLevel: 0.005
Experience: 0.007
LoanAmount: 0.038
LoanDuration: 0.008
MaritalStatus: 0.001
NumberOfDependents: 0.002
HomeOwnershipStatus: 0.002
MonthlyDebtPayments: 0.007
CreditCardUtilizationRate: 0.006
NumberOfOpenCreditLines: 0.003
NumberOfCreditInquiries: 0.002
DebtToIncomeRatio: 0.017
BankruptcyHistory: 0.003
LoanPurpose: 0.002
PreviousLoanDefaults: 0.003
PaymentHistory: 0.004
LengthOfCreditHistory: 0.008
SavingsAccountBalance: 0.005
CheckingAccountBalance: 0.005
TotalAssets: 0.017
TotalLiabilities: 0.005
UtilityBillsPaymentHistory: 0.005
JobTenure: 0.003
NetWorth: 0.017
MonthlyLoanPayment: 0.021
RiskScore: 0.609


In [106]:
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})
threshold = 0.01
important_features_df = importance_df[importance_df['Importance'] > threshold]

# Print the important features
print("Important Features:")
print(important_features_df)

Important Features:
               Feature  Importance
1         AnnualIncome    0.185742
5           LoanAmount    0.038353
14   DebtToIncomeRatio    0.017138
22         TotalAssets    0.016810
26            NetWorth    0.017103
27  MonthlyLoanPayment    0.021193
28           RiskScore    0.608513


In [107]:
selected_features = important_features_df['Feature'].values
X_important = X[selected_features]

In [108]:
X_important

,AnnualIncome,LoanAmount,DebtToIncomeRatio,TotalAssets,NetWorth,MonthlyLoanPayment,RiskScore
0,39948,13152,0.358336,146111,126928,419.805992,49.0
1,39709,26045,0.330274,53204,43609,794.054238,52.0
2,40724,17627,0.244729,25176,5205,666.406688,52.0
3,69084,37898,0.436244,104822,99452,1047.506980,54.0
4,103264,9184,0.078884,244305,227019,330.179141,36.0
...,...,...,...,...,...,...,...
19995,30180,24521,0.468077,80969,55327,905.767712,55.0
19996,49246,25818,0.317372,69571,64002,958.395633,54.0
19997,48958,37033,0.023014,108316,103663,945.427454,45.0
19998,41025,14760,0.534517,22085,10600,411.168284,59.0


In [119]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_important, y, test_size=0.3)

In [121]:
regressor = RandomForestRegressor(n_estimators = 200, random_state = 0)
regressor.fit(Xtrain, ytrain)
y_pred = regressor.predict(Xtest)

In [122]:
regressor.score(Xtrain, ytrain)
score = round(regressor.score(Xtrain, ytrain) * 100, 2)
print(f"Training score: {score}%")

Training score: 99.51%


In [123]:
from sklearn import tree

In [124]:
treeModel = tree.DecisionTreeClassifier()

In [125]:
treeModel.fit(Xtrain,ytrain)

DecisionTreeClassifier()

In [126]:
treeModel.score(Xtest,ytest)

0.987

In [127]:
from sklearn.linear_model import LogisticRegression
logRModel = LogisticRegression()

In [128]:
logRModel.fit(Xtrain,ytrain)

LogisticRegression()

In [129]:
logRModel.score(Xtest,ytest)

0.9213333333333333

In [130]:
import pickle 

with open('decision_tree_model.pkl', 'wb') as file:
    pickle.dump(treeModel, file)

print("Decision Tree model has been pickled successfully.")

with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(logRModel, file)

print("Logistic Regression model has been pickled successfully.")

with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(regressor, file)

print("Random Forest model has been pickled successfully.")

Decision Tree model has been pickled successfully.
Logistic Regression model has been pickled successfully.
Random Forest model has been pickled successfully.
